# 1. Radar
Por: Camila Lopes ([cclopes.netlify.app](https://cclopes.netlify.app/))  


Neste módulo, iremos aprender a lidar com dados de radares meteorológicos usando a linguagem Python, passando por leitura, visualização e processamento.


In [ ]:
# Importando as bibliotecas necessárias
import numpy as np

import pyart
import wradlib as wrl

## 1.1. Lendo dados de radar

### 1.1.1. Tipos de dados
Antes de aprender a ler dados de radar, é importante entender quais são os tipos de dados gerados por radares, já que eles podem ser configurados de diversas formas.

#### Escaneamento
O radar mede a energia da radiação eletromagnética retroespalhada pelos alvos no caminho de propagação do feixe de energia transmitido. Os sistemas de radar são frequentemente designados pelo comprimento de onda e banda de frequência em que operam. O comprimento de onda usado é fixo e pré-determinado na fabricação do radar, e depende da aplicação do radar (na meteorologia, medir nuvens ou precipitação). A configuração de escaneamento (i.e., como o radar enviará esta radiação no tempo e no espaço) depende do local em que o radar será instalado e do objetivo dele, i.e., monitoramento ou pesquisa.

Radares de monitoramento são, em geral, de bandas de frequência menor (2 a 8 GHz, Bandas S e C) pois sofrem menos atenuação. Para tirar mais vantagem ainda disso, suas configurações de escaneamento são do tipo **volumétricas**, ou seja, em função do azimute (plano) e elevação (altura). Os dados gerados por esse tipo de escaneamento são uma composição de **PPIs** (*Plan Position Indicator*), sendo um PPI para cada elevação, equivalente a uma matriz tridimensional em coordenadas polares de azimute e elevação. Visualizações desses PPIs podem ser feitas no "plano de elevação" (imagem de todos os azimutes em uma elevação fixa) ou "vertical" (imagem de todas as elevações em um azimute fixo). Para gerar produtos mais simples de serem interpretados, o formato PPI é convertido para **CAPPI** (*Constant Altitude Plan Position Indicator*), agora sim equivalente a uma matriz tridimensional cartesiana (plano horizontal e altura). As visualizações dos CAPPIs podem ser feitas no plano horizontal (altura fixa) ou vertical (eixo x ou y fixo e altura variada).

A resolução temporal de aquisição de varreduaras volumétricas varia bastante, dependendo principalmente da quantidade de elevações: dependendo do local a ser monitorado, uma quantidade menor ou maior de elevações é feita, com o objetivo final de ter uma resolução temporal razoável para monitoramento em tempo real (em geral essa resolução varia entre 5 e 15 minutos).

Radares de pesquisa costumam ser de frequência maior (acima de 8 GHz, Bandas X, K e W) pois, mesmo sofrendo mais atenuação, principalmente de gotas de chuva, são mais sensíveis à gotículas de nuvem e cristais de gelo, permitindo estudos de microfísica de nuvens. Além do escaneamento volumétrico, uma configuração interessante para esses estudos é do tipo **setorial**, em que uma região de azimutes é fixada, com resolução temporal podendo ser menor que 5 minutos. Este escaneamento pode ser feito por setores de PPIs ou no formato **RHI** (*Range Height Indicator*), onde o escanemento é feito variando a elevação em determinados ângulos azimutais. A visualização de RHIs em geral são feitas no "plano vertical" (distância em função da altura), mas também podem ser feitas no "plano de elevação" (todos os azimutes disponíveis em função de uma elevação).

Os radares de pesquisa das bandas K e W (acima de 20 GHz) sofrem ainda mais atenuação, de forma que outra configuração de escaneamento que é usada (dependendo da fabricação do radar, é a única configuração disponível) é do tipo **apontamento vertical**, em que, como o nome já diz, a antena fica apontada para cima (ou para baixo, se estiver a bordo de um avião ou satélite), medindo uma mesma região ao longo do tempo. A resolução temporal nessa configuração é na escala de segundos a poucos minutos. Os dados gerados por esse tipo de configuração são no formato **THI** (*Time Height Indicator*), equivalente a uma matriz bidimensional em função da altura e tempo, com a visualização nesse mesmo plano.

#### Formato
Existem diferentes formatos de arquivos que armazenam os tipos de dados explicados acima. O formato **binário** puro (extensão `.bin`, sem extensão, ou outra) é o formato mais simples de armazenamento. Ele pode ter diferentes tipos de estruturas (com cabeçalho ou não), mas não é possível saber essa estrutura apenas com o arquivo - é preciso ter algum README associado para conseguir ler o arquivo. Sabendo essa estrutura, é possível abrir esse tipo de arquivo com a função `open` do Python. Como exemplo [baseado neste artigo do Towards Data Science](https://towardsdatascience.com/loading-binary-data-to-numpy-pandas-9caa03eb0672), supondo que a estrutura do arquivo `filename.bin` é simplesmente uma matriz de um nível de CAPPI de 500 x 500 pontos, em que cada ponto é um `integer` de 4 bytes, o código abaixo lê esse arquivo e converte para uma matriz do `numpy`:  

```
# Carregando a biblioteca
import numpy as np

# Definindo o formato da matriz
dt = np.dtype([('x', '<i4'), ('y', '<i4')])

# Abrindo o arquivo
with open("filename.bin", "rb") as f:
    binary_data = f.read()

# Convertendo para a matriz
np_data = np.frombuffer(binary_data, dt)
```

Esse tipo de arquivo ainda pode ser encontrado para dados de radar brutos. Um exemplo é o **uf** (*Universal Format*, `.uf`). Este formato foi desenvolvido nos anos 1970 e era o mais comum para armazenamento e distribuição de dados de radar. Ele possui uma estrutura pré-definida com cabeçalho em caracteres ASCII e conteúdo em `integers` de 16 bits (saiba mais sobre ele no [documento do TITAN sobre o formato uf](https://ral.ucar.edu/projects/titan/docs/radial_formats/UfDoc.txt)). Como a estrutura é conhecida, é possível abrir esse tipo de arquivo no Python da mesma forma que dados binários, mas algumas bibliotecas como [`pyart`](https://arm-doe.github.io/pyart/index.html) possuem [funções que leem arquivos uf](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_uf.html?highlight=read%20uf). Outros formatos de arquivo estão associados aos softwares proprietários dos radares que geraram os dados, por exemplo, o formato [**MDV**](https://ral.ucar.edu/projects/titan/docs/MDV_XML_ICD.1.0.pdf), [**IRIS** (Vaisala)](https://www.vaisala.com/en/products/instruments-sensors-and-other-measurement-devices/weather-radar-products/iris-focus), [**GAMIC**](https://www.gamic.com/), [**RAINBOW**](https://www.leonardocompany.com/en/products/rainbow-5-application-software), entre outros. Em geral, as bibliotecas [`pyart`](https://arm-doe.github.io/pyart/index.html) e [`wradlib`](https://docs.wradlib.org/en/stable/index.html) que veremos a seguir tem suporte para eles.

Atualmente, os dados de radar usam formatos de dados científicos conhecidos para padronizar e eliminar a necessidade de fornecer descrições de estrutura de escrita binária:

- **HDF5** (*Hierarchical Data Format*, `.HDF5`): este formato é utilizado para armazenar grande quantidade de dados de quaisquer dimensões de forma mais organizada, com metadados associados. Diferente dos dados binários puros, não é preciso saber a estrutura dos dados. Em Python, é só usar a função `File()` da biblioteca [`h5py`](https://docs.h5py.org/en/stable/) para abrir esse tipo de arquivo. Algumas bibliotecas que processam dados no formato HDF5 usam essa biblioteca dentro de suas funções;

- **NetCDF** (*Network Common Data Form*, `.nc`): este formato foi desenvolvido nos anos 1980 pela [UCAR (*University Corporation for Atmospheric Research*)](https://www.ucar.edu/) especificamente para armazenar e compartilhar dados científicos. Atualmente na versão 4 [mantida pela Unidata](https://www.unidata.ucar.edu/software/netcdf/), e similar ao HDF5, o formato NetCDF armazena dados em quaisquer dimensões com metadados. Existe uma convenção para metadados de meteorologia chamada [CF (*Climate and Forecast*)](http://cfconventions.org/) com o objetivo de padronizar e facilitar o processamento de dados de diversas fontes; para dados de radar, essa convenção é chamada [CfRadial](https://github.com/NCAR/CfRadial). Em Python, é possível abrir esse tipo de arquivo diretamente com a biblioteca [`netCDF4`](https://unidata.github.io/netcdf4-python/netCDF4/index.html), mas diversas bibliotecas usam essa como base para processar dados NetCDF, atualmente sendo recomendável a biblioteca [`xarray`](http://xarray.pydata.org/en/stable/weather-climate.html);

- Outros: outros formatos de arquivo estão associados aos softwares dos radares que geraram os dados. As bibliotecas [`pyart`](https://arm-doe.github.io/pyart/index.html) e [`wradlib`](https://docs.wradlib.org/en/stable/index.html) que veremos a seguir tem suporte a diversos formatos como [**MDV**](https://ral.ucar.edu/projects/titan/docs/MDV_XML_ICD.1.0.pdf), [**IRIS** (Vaisala)](https://www.vaisala.com/en/products/instruments-sensors-and-other-measurement-devices/weather-radar-products/iris-focus), [**GAMIC**](https://www.gamic.com/), [**RAINBOW**](https://www.leonardocompany.com/en/products/rainbow-5-application-software), entre outros.  

Sabendo agora quais são os tipos de dados, podemos aprender como ler dados de radar com as bibliotecas Py-ART e wradlib.

A grande diferença entre as duas bibliotecas é a forma de lidar com os dados lidos:

- O Py-ART converte os dados lidos para estruturas pré-definidas (classes [`Radar`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html#pyart.core.Radar), [`Grid`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Grid.html#pyart.core.Grid), [`HorizontalWindProfile`](https://arm-doe.github.io/pyart/API/generated/pyart.core.HorizontalWindProfile.html#pyart.core.HorizontalWindProfile) ou [`RadarSpectra`](https://arm-doe.github.io/pyart/API/generated/pyart.core.RadarSpectra.html#pyart.core.RadarSpectra)), o que é **vantajoso para desenvolver funções (de visualização e processamento)** em cima dessas estruturas mas é **desvantajoso quando as funções de leitura de dados não tem suporte para diferentes configurações** (como veremos mais à frente);

- O wradlib tenta ao máximo manter a estrutura do arquivo original, com os dados em forma de matrizes multidimensionais ([`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html)) e os metadados em forma de dicionários, o que é **vantajoso para abrir arquivos com diferentes configurações** mas é **desvantajoso para desenvolver funções comuns para diferentes formatos de dados**.

> Existem outras bibliotecas em Python que lidam com dados de radar dentro da iniciativa [Open Radar Science](https://openradarscience.org/). Veja uma lista dessas bibliotecas (e outras aplicações) [aqui](https://openradarscience.org/projects/).

### 1.1.2. Py-ART

A biblioteca Py-ART possui dois módulos que leem dados de radar: [`pyart.io`](https://arm-doe.github.io/pyart/API/generated/pyart.io.html) e [`pyart.aux_io`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.html). Os dois módulos possuem diversas funções que leem os diferentes tipos de dados explicados acima.

#### [`pyart.io.read()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read.html#pyart.io.read)
Essa é a função mais básica (*que deve ser usada primeiro*). Ela basicamente identifica qual o tipo de arquivo e lê usando a função correspondente (entre as opções [`read_cfradial()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_cfradial.html#pyart.io.read_cfradial), [`read_chl()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_chl.html#pyart.io.read_chl), [`read_mdv()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_mdv.html#pyart.io.read_mdv), [`read_nexrad_archive()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_nexrad_archive.html#pyart.io.read_nexrad_archive), [`read_nexrad_cdm()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_nexrad_cdm.html#pyart.io.read_nexrad_cdm), [`read_nexrad_level3()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_nexrad_level3.html#pyart.io.read_nexrad_level3), [`read_rsl()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_rsl.html#pyart.io.read_rsl), [`read_sigmet()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_sigmet.html#pyart.io.read_sigmet) e [`read_uf()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_uf.html#pyart.io.read_uf) disponíveis atualmente) de forma que você não precisa usar especificamente uma dessas funções. 

Como exemplo, o código abaixo lê arquivos ([`pyart.io.read()`]) do radar de Presidente Prudente (SP) (do tipo Sigmet (IRIS)) e do radar da FCTH durante o experimento CHUVA-GLM (disponível no [FTP do CPTEC-INPE](http://ftp.cptec.inpe.br/chuva/glm_vale_paraiba/experimental/level_0/eq_radar/esp_band_s/st_cth/)) e mostra na tela as principais informações sobre eles. A forma mais rápida de verificar se o arquivo foi lido corretamente é usar a função [`[variável de radar].info()`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.info.html#pyart.core.Radar.info) para obter as informações dos dados. A saída dela mostra os atributos (variáveis, azimute, elevação, parâmetros do instrumento, metadados, etc) com o tipo de dado (`float`, `int`) e forma (quantidade de pontos e dimensões - por exemplo, a variável `reflectivity` tem 2 dimensões, 5760 x 957 pontos).

In [ ]:
# Lendo o arquivo
dados_pp = pyart.io.read("../dados/radar/PP/PPR161225194610.RAWK318")
dados_fcth_chuva = pyart.io.read("../dados/radar/FCTH/RADL04072020111101004500.uf")
# Outra opção, se você já souber o formato do arquivo, é a usar a função correspondente
# dados_pp = pyart.io.read_sigmet("./dados/radar/PP/PPR161225194610.RAWK318")

# Informações sobre o arquivo
print("Dados de Presidente Prudente:")
print(dados_pp.info(level="compact"))
print("---------------------------- \nDados da FCTH (CHUVA):")
print(dados_fcth_chuva.info(level="compact"))

#### [`pyart.aux_io.read_[...]()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.html)

Se você tentar usar a função `pyart.io.read()` para abrir um arquivo e não funcionar, quer dizer que o arquivo tem um formato diferente dos suportados por essa função, de forma que o próximo passo é usar uma das funções do módulo `pyart.aux_io`. As opções disponíveis atualmente são: [`read_d3r_gcpex_nc()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_d3r_gcpex_nc.html#pyart.aux_io.read_d3r_gcpex_nc), [`read_edge_netcdf()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_edge_netcdf.html#pyart.aux_io.read_edge_netcdf), [`read_gamic()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_gamic.html#pyart.aux_io.read_gamic), [`read_kazr()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_kazr.html#pyart.aux_io.read_kazr), [`read_kazr_spectra()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_kazr_spectra.html#pyart.aux_io.read_kazr_spectra), [`read_noxp_iphex_nc()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_noxp_iphex_nc.html#pyart.aux_io.read_noxp_iphex_nc), [`read_odim_h5()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_odim_h5.html#pyart.aux_io.read_odim_h5), [`read_pattern()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_pattern.html#pyart.aux_io.read_pattern), [`read_radx()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_radx.html#pyart.aux_io.read_radx), [`read_rainbow_wrl()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_rainbow_wrl.html#pyart.aux_io.read_rainbow_wrl) e [`read_sinarame_h5()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_sinarame_h5.html#pyart.aux_io.read_sinarame_h5). Em um cenário ideal, você terá alguma informação sobre o formato do arquivo que ajude a escolher a função correta, mas como esse nem sempre é o caso, se acostume com a ideia de tentar usar todas e de que nenhuma delas pode funcionar (o que costuma acontecer com radares usados aqui no Brasil).

Como exemplos, o código abaixo lê arquivos de dois radares diferentes (São Roque - SP e SIPAM - AM) que tem o mesmo formato GAMIC-HDF5 e mostra suas informações.

In [ ]:
# Lendo o arquivo
dados_sr = pyart.aux_io.read_gamic("../dados/radar/SR/SRO-250--2017-03-14--18-30-22.mvol")
dados_sipam = pyart.aux_io.read_gamic("../dados/radar/SIPAM/RADL08061720140301081200.HDF5")

# Informações sobre o arquivo
print("DADOS DE SÃO ROQUE")
print(dados_sr.info(level="compact"))
print("\nDADOS DO SIPAM")
print(dados_sipam.info(level="compact"))

#### Outros (se nada der certo...)

Se nenhuma das opções acima funcionar, é hora de investigar. É difícil encontrar um formato de dados de radar que não esteja incluído em uma das funções citadas anteriormente, de forma que o que geralmente gera erro é diferenças nos nomes dos atributos ou variáveis. Isso acontece porque essas funções supõem que o arquivo tem uma estrutura com certos nomes pré-definidos (para radares americanos), que podem ser diferentes dos configurados nos radares brasileiros. Por exemplo, ao tentar um arquivo do radar de apontamento vertical MIRA (ATTO-Campina - AM) (que possui configuração similar ao radar KAZR do ARM) com a função correspondente (código abaixo), é gerado um erro `KeyError` associado ao atributo de latitude. Nesse caso, o atributo no arquivo do MIRA não é uma variável chamada `'lat'` (como esperado pelo comando `latitude = cfradial._ncvar_to_dict(ncvars['lat'])`) e sim uma subclasse do objeto (`ncobj.Latitude`).

In [ ]:
pyart.aux_io.read_kazr("../dados/radar/MIRA_CAMPINA/20200306_0100.mmclx")

A solução para corrigir esse problema é modificar a função de leitura, adaptando para esse formato dos dados. Nesse caso, eu precisei modificar vários outros trechos a ponto de criar uma nova função, chamada `read_mira()`, que está no arquivo `read_mira_radar.py`. O código abaixo usa essa função.

In [ ]:
# Importando a função
from read_mira_radar import read_mira

# Lendo o arquivo
dados_mira = read_mira("../dados/radar/MIRA_CAMPINA/20200306_0100.mmclx")[0]

# Informações sobre o arquivo
dados_mira.metadata

Outros radares cujos dados não conseguem ser lidos pelas funções do Py-ART são o da FCTH (Salesópolis - SP) do Projeto SOS-CHUVA (dados disponíveis no [FTP do CPTEC-INPE](http://ftp.cptec.inpe.br/chuva/soschuva/experimental/level_0/eq_radar/esp_band_s/st_sales/)) e o XPOL do Projeto CHUVA (já instalado em [Manaus - AM](http://ftp.cptec.inpe.br/chuva/goamazon/experimental/level_2/eq_radar/esp_band_x/), [Campinas - SP](http://ftp.cptec.inpe.br/chuva/soschuva/experimental/level_2/eq_radar/esp_band_x/) e [São Borja - RS](http://ftp.cptec.inpe.br/chuva/relampago_sao_borja/X-Band_Radar_Brazil/), com dados disponíveis no FTP do CPTEC-INPE pelos links de cada localidade). Os dados "brutos" deles estão no formato RAINBOW (separados por variável), que podem ser convertidos para HDF5-GAMIC com um [conversor desenvolvido pelo Dr. Thiago Biscaro (DSA/CPTEC/INPE)](https://github.com/tsbiscaro/Rainbow5_to_HDF5). Para abrir esses dados no formato GAMIC, é necessário uma versão modificada da `read_gamic()`, chamada `read_rainbow_hdf5()`, que está no arquivo `read_brazil_radar_py3.py`. O código abaixo lê e mostra informações dos arquivos dos dois radares.

In [ ]:
# Importando a função
from read_brazil_radar_py3 import read_rainbow_hdf5

# Lendo os arquivos
dados_fcth_soschuva = read_rainbow_hdf5("../dados/radar/FCTH/PNOVA2-20170314182730.HDF5")
dados_xpol = read_rainbow_hdf5("../dados/radar/XPOL_CMP/117BRX-20171115215006.HDF5")

# Informações sobre os arquivos
print("DADOS DO FCTH")
print(dados_fcth_soschuva.info(level="compact"))
print("\nDADOS DO XPOL")
print(dados_xpol.info(level="compact"))

Mais um exemplo é de dados de CAPPIs do radar do SIPAM (AM) processados no período do GoAmazon (disponíveis no [repositório do ARM](https://adc.arm.gov/discovery/#/results/instrument_class_code::sipam-s-band-cappi)). O formato desses dados é NetCDF e o conteúdo é similar aos dados em grade gerados pelo Py-ART, que são lidos com a função [`pyart.io.read_grid()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_grid.htm). A função `read_sipam_cappi()` do arquivo `read_sipam_cappis.py` é uma adaptação dessa função original. O código abaixo lê o arquivo e mostra suas informações.

In [ ]:
# Importando a função
from read_sipam_cappis import read_sipam_cappi

# Lendo o arquivo
dados_sipam_cappi = read_sipam_cappi("../dados/radar/SIPAM/sbmn_cappi_20140301_081203.nc")

# Informações sobre o arquivo
dados_sipam_cappi.metadata

Um último exemplo comum em dados de radar reprocessados é de dados de CAPPIs do radar da FCTH durante o experimento CHUVA-GLM (disponíveis no [FTP do CPTEC-INPE](http://ftp.cptec.inpe.br/chuva/glm_vale_paraiba/experimental/level_2/eq_radar/esp_banda_s/st_cth/)). Eles estão no formato **binário**, na seguinte configuração de grade:  

- 15 x 500 x 500 pontos (eixos z, y e x, respectivamente)
- Resolução horizontal de 1 x 1 km
- Resolução vertical de 1 km, com o primeiro nível em 2 km de altura

O código abaixo abre o arquivo como um [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) e aplica algumas correções necessárias.

> *Agradecimentos a Franciane Rodrigues (EPL Informática, Projeto Alerta Raios) pela sugestão de dados!*

In [ ]:
dados_fcth_chuva_l2 = np.fromfile(
    "../dados/radar/FCTH/RADL24072120111101004500.dat", dtype="float32"
    ).reshape((15, 500, 500))
# - Excluindo dados acima de 100 dBZ
dados_fcth_chuva_l2[dados_fcth_chuva_l2 >= 100] = np.nan
# - A grade foi montada com o eixo y invertido
dados_fcth_chuva_l2 = np.flip(dados_fcth_chuva_l2, axis=1)
print(dados_fcth_chuva_l2)

### 1.1.3. wrablib

Da mesma forma que o Py-ART (mas sem uma "função padrão" como a `pyart.io.read()`), o wradlib possui [diversas funções que leem diferentes tipos de dados de radar](https://docs.wradlib.org/en/stable/notebooks/fileio/wradlib_radar_formats.html), com versões similares (como a [`read_gamic_hdf5()`](https://docs.wradlib.org/en/stable/generated/wradlib.io.hdf.read_gamic_hdf5.html#wradlib.io.hdf.read_gamic_hdf5) e a [`read_iris()`](https://docs.wradlib.org/en/stable/generated/wradlib.io.iris.read_iris.html#wradlib.io.iris.read_iris)) e suporte para outros formatos como o [DX e RADOLAN](https://docs.wradlib.org/en/stable/io.html#read-radolan-and-dx) (do Serviço Meteorológico Alemão). Como exemplo, o código abaixo lê os dados "brutos" no formato RAINBOW (separados por variável) do radar da FCTH durante o Projeto SOS-CHUVA e mostra suas principais informações. A dificuldade nesse caso é entender a estrutura do arquivo.

In [ ]:
# Lendo o arquivo
dados_fcth_dbz = wrl.io.read_rainbow("../dados/radar/FCTH/2017031418273000dBZ.vol", loaddata=False)

# Informações sobre o arquivo
print("Date: ", dados_fcth_dbz['volume']['@datetime'])
print("Type: ", dados_fcth_dbz['volume']['@type'])
print("Metadata:")
dados_fcth_dbz['volume']['scan']['pargroup']

A partir da [versão 1.12 do wradlib](https://github.com/wradlib/wradlib/releases/tag/1.12.0), algumas das funções citadas acima receberam suporte para o pacote [`xarray`](https://docs.xarray.dev/en/stable/), que facilita a manipulação de matrizes multidimensionais. O mesmo dado acima pode ser lido com a função [`open_radar_dataset()`](https://docs.wradlib.org/en/stable/generated/wradlib.io.xarray.open_radar_dataset.html) com a engine "rainbow". Veja como a estrutura fica totalmente diferente e facilita o acesso aos dados:

In [ ]:
dados_fcth_dbz = wrl.io.xarray.open_radar_dataset("../dados/radar/FCTH/2017031418273000dBZ.vol", engine="rainbow")

# Informações sobre o arquivo
print(dados_fcth_dbz)

# Vendo uma das elevações
print("Dados da primeira elevação:")
print(dados_fcth_dbz[0])